# Algoritmos de Depth-Limited Search y Iterative Deepening Search

El algoritmo de Depth-Limited Search o de profunidad limitada es una versión del algoritmo de Depth-First Search donde se limita la profundidad en el árbol de búsqueda. 

Aquí aplicamos este algoritmo al problema del mundo de la aspiradora y también extendemos este algoritmo al llamado Iterative Deepening Search que permitirá encontrar una solución sin depender de la profundidad.

### Preparación del problema

En primer lugar definimos el problema del mundo de la aspiradora.

In [1]:
from itertools import product
from VaccuumWorldSearchProblem import State, Transition, SearchProblem

cuadros = ['A','B']

#Inicializa contador de estados
i = 0
#Guarda estados
states = {}
#Determina los estados de suciedad
for dirt in product([1,0],repeat=len(cuadros)):
    #Determina la posición del agente
    for agent_loc in cuadros:
        #Asigna valores al objeto estado
        state = State()
        state.world = {cuadros[j]:dirt[j] for j in range(len(cuadros))}
        state.agent_location = agent_loc
        #Guarda el estado en diccionario
        states[i] = state
        #Aumenta el valor de i
        i += 1
        
#Definimos transiciones
transition_model = Transition()
transition_model.get_transitions(states)

#Definimos el problema de búsqueda
vacuum_problem = SearchProblem(states=states,goal=[6,7], initial=0,
                        transition_model=transition_model,actions=['Clean','Left','Right'])

print(vacuum_problem)

States: 
 -[0, 1, 2, 3, 4, 5, 6, 7]
Initial State:
 -0
Final states:
 -[6, 7]
Actions:
 -['Clean', 'Left', 'Right']
Transition model:
 -[(0, 'Clean', 4), (0, 'Right', 1), (1, 'Clean', 3), (1, 'Left', 0), (2, 'Clean', 6), (2, 'Right', 3), (3, 'Left', 2), (4, 'Right', 5), (5, 'Clean', 7), (5, 'Left', 4), (6, 'Right', 7), (7, 'Left', 6)]


### Funciones de apoyo

Los algoritmos de Depth-Limited Search e Iterative Deepening Search utilizan una pila de tipo LIFO (Last-in-First-Out) que revisa los elementos desde el útlimo añadido hasta el primero, definimos esta pila a continuación:

In [2]:
class LIFOQueue(object):
    """
    Clase de una cola LIFO en los problemas de búsqueda.
    """
    def __init__(self):
        self.queue = []
  
    def __str__(self):
        return ' '.join([str(q) for q in self.queue])
  
    def isEmpty(self):
        """
        Returns
        -------
        empty : bool
            Devuelve el valor True si el queue está vacío
        """
        return self.queue == []
  
    def push(self, element):
        """
        Arguments
        ---------
        element : 
            El elemento que se va agregar al queue
        """
        self.queue.append(element)
  
    def pop(self):
        """
        Elimina el elemento con más valor según un peso f
        y regresa el elemento correspondiente a este valor.
        """
        #Regresa el primer elemento que llegó
        first_element = self.queue[-1]
        #Borra este elemento de la cola
        del self.queue[-1]
    
        return first_element
    
    def top(self):
        """
        Devuelve el elemento en el tope de la pila.
        A diferencia de pop, no elimina el elemento de la pila.
        """
        #Tope de la pila LIFO
        item = self.queue[-1]
        
        return item

Los objetos nodos se definirán por el estado, el padre, la acción, y además el elemento de profundidad que indicará cuál es el elemento con mayor profundidad en el árbol de búsqueda.

In [3]:
class Node(object):
    """
    Clase para crear nodos con sus atributos.
    """
    def __init__(self):
        self.state = 0
        self.parent = None 
        self.action = None
        self.depth = 0
        
    def __str__(self):
        if self.parent == None:
            return "State: {}, Cost: {}".format(self.state,self.depth)
        else:
            return "State: {}, Action: {}, Parent: {}, Depth: {}".format(self.state,self.action,self.parent.state,self.depth)

La función para expandir los nodos guardará la profundidad del nodo. El nodo raíz tendrá profundidad 0, y por cada expansión se sumará 1 a la profundidad. Los elementos de mayor profundidad se irán agregando a la pila, de tal forma que los elementos de mayor profundidad estén en el tope de ésta.

In [4]:
def expand(problem, node):
    """
    Función para expandir los nodos dado el problema.
    
    Argumentos
    ---------
    problem : objeto
        Problema de búsqueda.
    node : objeto
        Nodo que se va a expandir.
        
    Salida
    ------
    childs :generator
        Generador con los hijos del nodo.
    """
    #Nodo en el que se inicia
    s = node.state 

    for action in problem.actions:
        #Ejecuta la acción
        new_s = problem.act(s, action)
        #Guarda la profundidad del nodo
        depth = node.depth + 1
        
        #Genera un nuevo nodo
        new_node = Node()
        new_node.state = new_s
        new_node.parent = node 
        new_node.action = action 
        new_node.depth = depth

        yield new_node

Finalmente, definiremos una función para revisar si un nodo <b>forma un ciclo</b>. Esta función revisará si los nodos ya han sido visitados, si lo han sido entonces forman un ciclo.

In [5]:
def CheckVisited(node, visited, recStack):
        #Marca el nodo actual como visitado
        visited[node.state] = True
        #Añade al stack de recursión
        recStack[node.state] = True
 
        #Si hay un padre hacer...
        if node.parent != None:
            #Revisa si el padre ha sido visitado
            if visited[node.parent.state] == False:
                #Aplica una recursión
                if CheckVisited(node.parent, visited, recStack) == True:
                    return True
            elif recStack[node.parent.state] == True:
                return True
 
        #Quita el nodo de la recursión
        recStack[node.state] = False
        
        return False

def isCycle(node,nodes):
    #Crea lista de False por cada nodo
    visited = [False]*(len(nodes)+1)
    recStack = [False]*(len(nodes)+1)
    #Revisa los nodos que se han visitado
    for prev_node in nodes:
        if CheckVisited(node, visited, recStack) == True:
            return True
    return False

## Algoritmo Depth-Limited Search

Definimos el algoritmo de Depth-Limited Search. El algoritmo regresará los nodos que ha encontrado en el camino de búsqueda o bien señalará que no ha encontrado el camino ("failure"). Si el método no es capas de encontrar un nodo final al agotar la máxima profundidad, regresará un "cutoff".

In [6]:
def DepthLimitedSearch(problem,l):
    """
    Algoritmo Depth-Limited Search para generar el camino más apto
    en el árbol de búsqueda de un problema.
    
    Argumentos
    ----------
    problem : objeto
        Problema de búsquda.
    l : int
        Máxima profunidad del árbol de búsqueda.
        
    Salida
    ------
    nodes : list
        Lista de los nodos que llevan a la solución.
    """
    #Almacenamiento de nodos
    nodes = []
    #Nodo inicial
    node = Node()
    node.state = problem.initial    
    #Frontera con cola de prioridad
    frontier = LIFOQueue()
    frontier.push(node)
    #resultado
    result = "failure"

    #Mientras la frontera no esté vacía
    while frontier.isEmpty() == False:
        #Pop en frontera
        node = frontier.pop()
        #Guarda el nodo en la lista
        nodes.append(node)
        
        if node.state in problem.goal:
            return nodes
        
        if node.depth > l:
            result = "cutoff"
        elif isCycle(node,nodes) == False:
            for child in expand(problem, node):
                frontier.push(child)
        
    return result

Aplicamos el algoritmo al problema del mundo de la aspiradora. En este caso, el encontrar una solución depende de la profundidad máxima que determinemos.

In [7]:
#Aplicamos el algoritmo al problema
solution = DepthLimitedSearch(vacuum_problem,1)

print(solution)

cutoff


## Iterative Deepening Search

Para solucionar el problema de los "cutoff", utilizaremos el algoritmo de Iterative Deepening Search que lo que hace es iterar sobre la máxima profundidad y aplicar el método de Depth-Limited Search. Es decir, explorará las profundidades desde 0 hasta infinito hasta encontrar una en donde se llegué a una solución del problema.

In [8]:
def IterativeDeepeningSearch(problem):
    """
    Algoritmo de Iterative Deepening Search.
    
    Argumentos
    ----------
    problem : object
        Problema del que buscamos la solución.
    """
    #Inicializa la profundidad
    l = 0
    #Revisa si hay resultados
    result = DepthLimitedSearch(problem,l)
    #Itera hasta encontrar una solución
    while result == "cutoff":
        #Agrega una profundidad más
        l += 1
        #Revisa el resultado
        result = DepthLimitedSearch(problem,l)
    
    print("Resuelto en {} iteraciones".format(l))
    return result

In [9]:
#Aplicamos el algoritmo al problema
solution = IterativeDeepeningSearch(vacuum_problem)

for node in solution:
    print(node)

Resuelto en 2 iteraciones
State: 0, Cost: 0
State: 1, Action: Right, Parent: 0, Depth: 1
State: 1, Action: Right, Parent: 1, Depth: 2
State: 0, Action: Left, Parent: 1, Depth: 2
State: 3, Action: Clean, Parent: 1, Depth: 2
State: 3, Action: Right, Parent: 3, Depth: 3
State: 2, Action: Left, Parent: 3, Depth: 3
State: 3, Action: Clean, Parent: 3, Depth: 3
State: 0, Action: Left, Parent: 0, Depth: 1
State: 4, Action: Clean, Parent: 0, Depth: 1
State: 5, Action: Right, Parent: 4, Depth: 2
State: 5, Action: Right, Parent: 5, Depth: 3
State: 4, Action: Left, Parent: 5, Depth: 3
State: 7, Action: Clean, Parent: 5, Depth: 3
